# Transfer Learning 2

## 1. Domain Adaptation

`-` Transfer Learning

* Supervised to Supervised
* Supervised with few labels to Supervised with many labels -> 얻을 수 있는 데이터의 양의 한계가 있을 때 (e.g., simulation 정보만 제공되는 상태에서 학습 후 현실에 적용: 시뮬레이션은 제한이 없으니까...)
* Unsupervised to Unsupervised -> PCA 변환 행렬을 다른 데이터에 그대로 사용할 수 있다? 라는 느낌

> Domain Adaptation Problem

`-` Examples

* 저해상도의 이미지와 국소 지역 라벨들 -> 레이블링 되지 않은 고해상도 이미지
* 여러 라벨의 시뮬레이션 이미지 -> 적은 라벨의 현재 이미지

### Mathematical formulation

$$\begin{align} & p_t(x) \neq p_s(x) \\
& p_t(T(x)) = p_s(T(x)) \\
& p_t(y | T(x)) = p_s(y | T(x))
\end{align}$$

$p_t,~ p_s$가 타겟/소스 도메인의 분포이고, $T$를 변환이라고 할 때.

* 타겟과 소스의 분포를 일치시키는 변환 $T$를 찾는다.

* 서로 다른 분포의 소스/타겟 도메인이 존재할 때, $T$라는 변환은 각 소스의 데이터 x에 대하여 $T(x)$가 두 도메인에 대해 같은 분포를 가지도록 만든다.
* 해당 변환을 찾을 수 있다면, 각 도메인에서 T(x)가 주어졌을 때 y에 대한 조건부 확률도 동일해야 한다. -> 변환하여 분류할 수 있는 형태로 주어진다.

> 현실에선 마지막은 이상적인 경우임. $T$를 찾기도 어려움. $T$를 찾고, 마지막 조건에 근접하도록 regularization.

### 왜 분포를 같게 만들려 하는가?

* X가 주어졌을 때 y의 확률이 어떻게 되는지를 알아야 지도학습 수행 가능
* 두 데이터가 완전히 일치하지 않더라도, 하나의 확률 분포를 가지고 있다면 $\to$ 동일한 예측을 수행할 수 있음
* 동일한 데이터: feature value가 똑같다는 게 아니라, 분포가 동일하다는 의미로써 언급

`-` 변환 후 분포의 동일성을 어떻게 파악할 것인가?

* 둘 다 정규분포를 따른다고 가정한 뒤, 평균과 분산을 맞춤: 정규분포가 아닌 데이터에 일반화 불가능
* KL-divergence, MMD 등을 사용함

### Metric Learning

1. 두 점 사이의 거리를 고려 $x_s^i,~ x_t^i$

$$d_W (x_s^i, x_t^i) = (x_s^i - x_t^i)^{\top} W (x_s^i - x_t^i)$$

> 유클리디안 거리의 제곱 수식에 $W$를 넣음.

2. $W$는 $W^{\frac12}$로 나타낼 수 있으므로, 위 수식은 $W^{\frac12}x_s^i$와 $W^{\frac12}x_t^i$간 거리. 각 점을 변환한 상태에서의 변환이라고 말할 수 있음. (선형 변환)

3. 선형으로 가는 조건에서, $x_s^i,~ x_t^i$의 레이블이 같다면: 변환 후 거리가 짧아야 함
    
    레이블이 다르다면: 변환 후 거리가 멀어야 함 -> 3번 조건을 최대한 만족시키는 방향

`1.` Semi-supervised domain adaptation

> 레이블이 같으면 거리가 $u$(upper)를 넘기지 않고, 레이블이 다르면 거리가 $l$(lower)보다 긴 변환이여야 함.
>
> 어떤 값을 정하는지는 자유...

`2.` positive-metrix $W$

arg min Tr(W) - log det(W)

### Asymmetric Transformations

1. Using similarity: inner product-based

$$\text{sim}_W (x_s^i, x_t^i) = {x_s^i}^{\top} W x_t^i$$

> 내적의 값이 클수록 유사하지 않은 것

2. Loss function을 고려

> 0보다 작을 수 없는 loss function -> 작아지기 위해서는 같은 레이블일 경우 $l$보다 큰 유사도 / 다른 레이블일 경우 u보다 작은 유사도
>
> $W$의 값 자체가 너무 커지는 것을 방지하기 위해서 Frobenius norm을 사용

하지만 위의 두 방법은 모두 하나의 행렬 $W$를 기반으로 한 선형 방법임 -> 변환의 표현력이 너무 단순함 -> 복잡한 변환을 생각할 필요가 있음 (초기의 방법임. 어떤 문제는 잘 안될 수 있음)

## 2. Maximum Mean Discrepancy

`-` Moment

* $X, Y$의 $1, 2, \cdots, m$차 표본적률까지 동일한지를 검사. 같다고 할 수 있을까? m을 적당히 끊으면 애매함. 몰루
* $m \to \infty$로 확장하면 같다고 할 수 있음. (존재하고, 수렴한다는 조건 하에서)

`-` MMD (Maximum Mean Discrepancy)의 아이디어

* 어떤 특정한 함수 공간 $H$에 대하여 $f \in H \Rightarrow \underset{f \in H, ~ ||f|| ≤ 1}{\sup} ~ E[f(X)] - E[f(Y)] \to 0$이면, X와 Y의 분포는 동일하다.
* $H$ (RKHS Reproducing Kernel Hilbert Space)에 포함되어 있는 함수의 수가 가산 무한일 수 있음

> exponential 형태의 kernel을 가져다 쓰면, 분포의 동질성을 식별할 수 있으며, 학습을 통해 두 분포를 alignment할 수 있음

`-` Kernel Trick: Inner product - Kernel

$$<\phi (x), \phi (y)> = K(x, y)$$

> 모든 커널은 커널과 연관된 $\phi$라는 변환이 존재하며, 이는 ???
>
> summation 차원에서 $K(x_i, x_j) = K(x_i, y_j) = K(y_i, y_j)$이면, $MMD(X, Y)$의 값이 0이 된다.

`-` Sample Reweighting

> Loss function: $\beta_i$를 곱하여 $x_s^i$샘플의 가중치를 조정. 아직 y가 없어서 두 데이터의 분포만 맞춤

`-` Sample Selection

> Loss function에서 각 샘플의 weight를 조정함에 따라, 손실함수에서 $x_s^i$의 영향력을 $x_t^i$와 맞춰줌
>
> $\alpha_i$를 $x_s^i$에 곱하되, 새로운 weight을 지정해도 $y$ 값의 비율이 바뀌지 않도록 보정 $\to$ 학습에 영향을 주지 않도록 만듦

classification 문제에서는 y값의 레이블 분포가 어떤지에 따라, 정확도가 크게 바뀜 $\to$ 변환 이후 전혀 다른 상황에서 학습을 하게 됨

`-` Domain Invariant Projection (DIP)

* 선형 변환된 것을 한번 더 MMD로 재변환. $W_x \to \phi(W_x)$
* 모형 자체가 비선형인 경우(DNN 등)가 들어오면, 선형 변환이 깨짐. 따라서 비선형으로 변환하여 성능을 유지

`-` Correlation approach

* de-correlate and re-correlate.
* 정규분포를 벗어나면 어떻게 될지 잘 모름

### DA 정리

* 두 개의 분포를 같게 만든다.
* 변환을 하고, 변환시의 y의 label을 같게 만든다.
* DNN의 어느 레이어에서 해야 할지는 완벽히 정해지지 않음 $\to$ Input 수준에서 수행하는 것을 가정

## 3. Distillation

### Idea

* 학생과 선생 구조
* 학생이 선생을 따라가게 만드는 구조로 설계하면 어떨까

`-` ReLU activation function

* activation이 0일 경우, 정보가 넘어가지 않음 $\to$ 그 뒤로도 연결이 계속 끊어짐. 불필요한 파라미터가 많음
* 뉴런 간 connection이 대다수 유지되지 못함. 많이 끊어짐.

`-` 모델 아키텍쳐의 요약

* 고성능의 대형 모델(Teacher)과 비슷한 결과가 나오는 작은 모델(Student)을 만들 수 있지 않을까?
* 전체 파라미터 구조와 학습하는 데이터의 규모가 작음. Teacher의 모든 지식을 전수받을 순 없으나, 꽤나 높은 성능을 발휘함

> 학습에 있어 어느 부분이 핵심 정보를 가지고 있는가? 에 대한 파악이 필요.
>
> 어떤 레이어가 가장 많은 정보? 층 자체가 student와 유사한 패턴을? relation?

`-` Knowleage Distillation

* 기존 고성능 모델보다 간단한 아키텍쳐로 조금 성능이 떨어지는 고효율의 모델을 구성

### **Methodology**

`-` teacher와 student가 산출하는 확률 벡터를 가능하면 동일하게 만들도록 함: Distillation Loss

* Response-based: 타겟 기반. $y$에 대한 예측의 쌍이 맞도록(확률 벡터가 유사하도록)
* Feature-based: First Layer vector $\to$ Input, Attribute. 중간 레이어의 변환값 $\to$ Feature. 확률 벡터 산출 이전의 feature layer 값이 유사하도록 (보통 바로 전 레이어 netout)
* Relation-based: 위 두 방식은 정보가 말단 레이어에 몰려있을 것이라고 생각함 $\to$ Feature가 다음 레이어로 넘어가는 과정이 유사하도록 - 레이어 간 correlation을 유사하게 만듦

> sota는 없음. 상황에 따라 다름

`-` Response-based approaches (확률값이므로 유사성 판별을 따로 고려)

* KL divergence: teacher와 student 간 확률이 동일하도록 만드는 loss function을 택함. 기존 손실에 추가

`-` Feature-based

* 단순히 feature vector 간 거리로 판단 가능. 하지만 두 벡터 간 스케일이 다르기 때문에 비교 이전에 norm으로 나누어 스케일을 조정한 뒤 비교해야 함
* 상수와 norm으로 스케일 조정

`-` Relaetd-based

* 2개 레이어 간 inner product가 유사한 정도 (스케일 조정)

`-` Other categories

* Online/Offline: Online Learning $\to$ 새로 들어온 데이터를 추가했을 때 어떻게 바꿔야 수학적으로 엄밀한지를 분석하여 조정. 새로 들어온 데이터 단독으로 모델을 구성하여 가중치를 변경
> convex에서는 잘 작동함. non-convex 문제에서는 이론적인 background는 없음. convergence가 얼마나 잘 되는지 정도에 국한함.
* self-distillation: 자기 자신이 teacher이자 student
* multi-teacher: 여러 teacher를 사용
* adversatial approach: 적대적 모형을 동시에 활용

## 4. 실습

### Temperature scale

`-` Overconfidence

* $f_1, f_2, \cdots, f_k$: 각 클래스에 대한 netout
* 보통 한 개의 클래스에 대한 확률만 1에 가까워지고, 나머지는 다 0에 가까운 경우가 많음
> 분류 확률이 실제와 동일하지 않더라도, prediction에서는 동일하게 잘 분류할 수 있음
>
> 확신의 정도... 사고 위험을 detection $\to$ 확률이 높을 수록 강한 위험을 주도록 만들 수 있음.
>
> 확률을 맞추는 문제 $\to$ Calibration. 확률 벡터 자체가 잘못되어 있다면, Distilation에서 해당 확률 벡터로 보정하는 것이 문제가 있다.

`-` Temperature scale로 보정...

$$\frac{\exp (f_i / \tau)}{\sum_i \exp (f_i / \tau)}$$

> 미묘하게 조정. 

### Cosine Loss

`-` 두 벡터가 같은 방향으로 향할 때, 값이 커짐

* 방향이 같다면 값이 0이 되고, 방향이 다르면 값이 -1이 되도록 음수를 취함
* 유클리디안 norm은 방향이 같아도 0이 아닐 수 있음 (완전히 동일해야 0이 됨) $\to$ 길이를 고려할 것일지를 생각함. 보통 DNN에서는 스케일이 들쭉날쭉하므로 Cosine Loss가 더 적합해 보임 ㅇㅇ